In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

# Helper libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from matplotlib.colors import ListedColormap
from keras.callbacks import EarlyStopping
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
pickle_path = '/Users/jbaron776/Downloads/Project/pickledDf.pkl'
# Unpickle data file
df = pd.read_pickle(pickle_path)

# Split the dataframe to Wake, REM and NREM dataframes based on labels
#wake_df = df[df['Label'] == 0] # Size: 6376080 rows for 4 channels
#rem_df = df[df['Label'] == 1] # Size: 39310560 rows for 4 channels
#nrem_df = df[df['Label'] == 2] # Size: 23854560 rows for 4 channels

# Setup training and testing set (80% Training / 20% Testing)
#train_df, test_df = train_test_split(df, test_size=0.2)

# [TRAINING]
# Setup Label dataframe 
#train_label_df = train_df.filter(like='Label')
# Setup Data dataframe
#train_data_df = train_df.drop(['Label'], axis=1)

# [TESTING]
# Setup Label dataframe 
#test_label_df = test_df.filter(like='Label')
# Setup Data dataframe
#test_data_df = test_df.drop(['Label'], axis=1)

# train_df Size: 55632960 
# test_df Size: 13908240
#print (train_data_df) 
#print(train_label_df)
#print (test_data_df)
#print(test_label_df)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jbaron776/Downloads/Project/pickledDf.pkl'

In [ ]:
# Create input features and target variables
X = df.iloc[:,1:4]
y = df.iloc[:,-1]

# Standardize the input feature so scale from -1 to 1
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
# Split input features and target variables into training dataset and test dataset. 
# Training Set: 80% Testing Set: 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [30]:
print(y_test.shape)

(13908240,)


In [ ]:

print('making the classifier...')
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=8)
print('classifier made')
knn.fit(X_train, y_train) 
print('data fit')


cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA','#00AAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00','#00AAFF'])
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
np.arange(y_min, y_max, 0.02))

# predict class using data and kNN classifier
Z = knn.predict(X_test)
print(Z)


making the classifier...
classifier made


In [1]:
# Put the result into a color plot

plt.figure()

# Plot also the training points
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cmap_bold)
plt.xlim(X_test.min(), X_test.max())
plt.ylim(y_test.min(), y_test.max())
plt.title("3-Class classification (k = 3)")
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
fig = plt.subplots(figsize=(5,10))
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) of Sleep Classification using CNN')
plt.legend(loc="lower right")
plt.show()